# Imports

In [115]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sb
import time
# import mat4py
import scipy.io 
import sklearn
from sklearn import preprocessing
import plotly.express as px
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans 
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.fft import fft, ifft
from scipy.signal import lfilter,butter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [116]:
!pip install tensorflow
!pip install antropy

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [117]:
import antropy as ant
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# from keras.utils import to_categorical

In [118]:
from tensorflow.keras.utils import to_categorical

# Data loading

In [217]:
! ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
real_signal_filename = 'Dataset1.mat'
attack_signal_filename = 'sampleAttack.mat'


In [5]:

def load_realsignal_from_path(fpath):
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['Raw_Data']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            total_rows.extend(np.array_split(raw_data[i][j], 4))
    raw_df = pd.DataFrame(total_rows)
    return raw_df

def load_attacksignal_from_path(fpath):
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['attackVectors']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            for k in range(raw_data.shape[2]):
                total_rows.append(raw_data[i][j][k])
    raw_df = pd.DataFrame(total_rows)
    return raw_df

In [6]:
real_df = load_realsignal_from_path(real_signal_filename)

In [7]:
attack_df = load_attacksignal_from_path(attack_signal_filename)

In [222]:
real_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,-12,-19,-30,-53,-65,-59,-46,-42,-65,-74,...,-49,-33,-40,-31,-45,-65,-43,-32,-23,-22
1,-4,23,39,23,22,5,8,26,26,10,...,-48,-33,-45,-51,-27,-31,-42,-63,-65,-39
2,-31,-48,-67,-88,-88,-49,-34,-36,-22,-23,...,6,2,1,12,11,3,48,70,68,55
3,73,54,69,66,37,19,27,54,45,1,...,1,44,61,64,36,28,16,26,11,-5
4,-63,-40,-15,-24,-15,-26,-45,-33,-42,5,...,-30,-33,-45,-56,-49,-48,-63,-65,-63,-54


# Data Normalization

In [11]:
def min_max_norm(df):
    x = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_minmax = pd.DataFrame(x_scaled)
    return df_minmax

In [8]:
def std_norm(df):
    x = df.values
    std_scaler = preprocessing.StandardScaler()
    x_scaled = std_scaler.fit_transform(x)
    df_std = pd.DataFrame(x_scaled)
    return df_std

In [9]:
x_superset = pd.concat( [std_norm(real_df), std_norm(attack_df)] , axis = 0 )

In [10]:
y_superset = [1.0 for i in range(len(real_df)) ]
y_superset.extend([0.0 for i in range(len(attack_df))])

# Test Train split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x_superset, y_superset, test_size=0.3, random_state = 1)

# Features extraction list

Features: PCA, Entropy, FFT Theta band, Bio Orthogonal Family, TF-IDF

models: SVM, Cosine, Kmeans, KNN, CNN

## PCA feature extraction

In [16]:
best_feature_count = 100
pca = decomposition.PCA(n_components = best_feature_count) # only keep two "best" features!
X_train_pca = pca.fit_transform(X_train) # apply PCA to the train data
X_test_pca = pca.transform(X_test)

In [91]:
X_test_pca

numpy.ndarray

## Entropy

In [119]:
def entropy_features(X):
    res_data=[]
    for i in range(len(X)):
        x = X[i]
                
        perm = ant.perm_entropy(x, normalize=True)
        spectral = ant.spectral_entropy(x, sf=160, method='welch', normalize=True)
        svd = ant.svd_entropy(x, normalize=True)
        approx = ant.app_entropy(x)
        sample = ant.sample_entropy(x)
        temp_ans = [perm, spectral, svd, approx, sample]
        res_data.append(temp_ans)
    return res_data

X_train_entropy = entropy_features(np.asarray(X_train))

X_test_entropy = entropy_features(np.asarray(X_test))


In [120]:
X_train_entropy_array = np.asarray(X_train_entropy)
X_test_entropy_array = np.asarray(X_test_entropy)

X_test_entropy_array.shape











(954, 5)

[0.999527119133609,
 0.9925346319571211,
 0.9999527254102225,
 1.6539396421530288,
 1.6162925158846508]

In [90]:
X_train

pandas.core.frame.DataFrame

## FFT theta band

In [13]:

def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut /nyq
    high = highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    #print(b,a)
    y = lfilter(b, a, data)
    return y
temp = X_train.values
temp1 = X_test.values

X_train_theta = []
for i in range(len(temp)):
    
    fft1 = fft (temp[i])
    theta1 = butter_bandpass_filter(fft1, 4.1, 8.0, 160)
    X_train_theta.append(theta1)

X_test_theta = []
for i in range(len(temp1)):
    
    fft1 = fft (temp1[i])
    theta1 = butter_bandpass_filter(fft1, 4.1, 8.0, 160)
    X_test_theta.append(theta1)

X_train_theta


[array([ 0.02630518+0.j        ,  0.09947827-0.00975682j,
         0.21490286-0.01326805j, ..., -5.42269282+0.71743208j,
        -5.8989683 +0.38313854j, -6.08471166+0.10667948j]),
 array([-0.31626082+0.j        , -1.34931399-0.27470233j,
        -2.60062785-1.40872434j, ..., -0.20277418-9.79621907j,
         4.19852449-9.4625055j ,  7.79565407-6.7945178j ]),
 array([17.01523406 +0.j        , 51.10704335 +3.55377711j,
        68.17225657+11.65039817j, ..., 86.79263465+37.55133576j,
        65.11007743+23.56949643j, 31.05366994 +2.54501869j]),
 array([  1.0104209 +0.00000000e+00j,   4.29740871+8.61508790e-03j,
          8.33838549-3.71657176e-02j, ..., -20.65346215+1.89064447e+01j,
        -21.73457759+2.70408588e+01j, -21.18166423+3.24537030e+01j]),
 array([  11.64557766 +0.j        ,   56.30287435-25.22628988j,
         115.99142112-99.33738974j, ...,  -34.9945359 +38.36282966j,
         -72.60830704+31.06802554j, -102.57971467+49.25914336j]),
 array([ 1.72272749 +0.j        , 10.7795

## Bio orthogonal family

## TF -IDF

In [84]:
X_train_tfidf = X_train.values.tolist() 
X_train_tfidf_rounded = []
for i in range (len(X_train_tfidf)):
    temp = []
    for j in range (len(X_train_tfidf[i])):
        a = round(X_train_tfidf[i][j],2)
        temp.append (a)
    X_train_tfidf_rounded.append(temp)

X_train_tfidf_rounded_string = []
for i in range (len(X_train_tfidf_rounded)):
    temp = []
    for j in range (len(X_train_tfidf_rounded[i])):
        a = str(X_train_tfidf_rounded[i][j])
        temp.append (a)
    newtemp = " ".join(temp)
    X_train_tfidf_rounded_string.append(newtemp)


# for i in range (len(X_train_tfidf)):
#     X_tfidf_string = str (X_train_tfidf[i])
# # documentA = 'the man went out for a walk'
# # documentB = 'abcd'
# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(X_train_tfidf)
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# df = pd.DataFrame (denselist, columns=feature_names)
# cvec = CountVectorizer()
# cvec_counts = cvec.fit_transform(X_train_tfidf)















In [108]:
X_test_tfidf = X_test.values.tolist()
X_test_tfidf_rounded = []
for i in range (len(X_test_tfidf)):
    temp = []
    for j in range (len(X_test_tfidf[i])):
        a = round(X_test_tfidf[i][j],2)
        temp.append (a)
    X_test_tfidf_rounded.append(temp)
 
X_test_tfidf_rounded_string = []
for i in range (len(X_test_tfidf_rounded)):
    temp = []
    for j in range (len(X_test_tfidf_rounded[i])):
        a = str(X_test_tfidf_rounded[i][j])
        temp.append (a)
    newtemp = " ".join(temp)
    X_test_tfidf_rounded_string.append(newtemp)

In [109]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X_test_tfidf_rounded_string)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
X_test_tfidffeatures = pd.DataFrame (denselist, columns=feature_names)

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [106]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X_)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
X_train_tfidffeatures = pd.DataFrame (denselist, columns=feature_names)

1

,01,02,03,04,05,06,07,08,09,10,...,89,91,92,93,94,95,96,97,98,99
0,0.108509,0.128983,0.133077,0.126935,0.135125,0.133077,0.141267,0.110556,0.106462,0.0,...,0.089827,0.052671,0.093456,0.067403,0.110431,0.072382,0.101983,0.085102,0.073513,0.083467
1,0.282291,0.214332,0.266608,0.231757,0.216074,0.209104,0.214332,0.181224,0.172511,0.0,...,0.050298,0.095771,0.091780,0.081954,0.071514,0.090355,0.053733,0.062085,0.062568,0.058504
2,0.122856,0.126951,0.122856,0.133094,0.112618,0.102380,0.104427,0.094189,0.133094,0.0,...,0.061468,0.093383,0.083881,0.091486,0.093638,0.077217,0.106853,0.089976,0.093128,0.095754
3,0.120606,0.095653,0.114368,0.122685,0.124765,0.139321,0.099812,0.116447,0.120606,0.0,...,0.100837,0.097265,0.102222,0.075793,0.075587,0.075966,0.076452,0.083966,0.069687,0.079788
4,0.116985,0.094409,0.090304,0.114932,0.094409,0.151875,0.110828,0.137508,0.106723,0.0,...,0.085308,0.081600,0.115305,0.082047,0.101075,0.096746,0.114404,0.077999,0.100714,0.113203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.191796,0.193677,0.161711,0.133505,0.165471,0.146668,0.163591,0.186155,0.154189,0.0,...,0.030395,0.028585,0.019808,0.019898,0.026458,0.022159,0.026761,0.015632,0.022506,0.011273
2222,0.187670,0.218702,0.224613,0.221658,0.218702,0.205403,0.200970,0.218702,0.180282,0.0,...,0.000000,0.001728,0.001730,0.001737,0.003465,0.000000,0.003505,0.000000,0.000000,0.003544
2223,0.197324,0.145501,0.143508,0.155467,0.145501,0.141515,0.155467,0.141515,0.115604,0.0,...,0.066738,0.039623,0.058323,0.058589,0.042069,0.035233,0.042551,0.042609,0.062025,0.043019
2224,0.222717,0.245389,0.233386,0.218716,0.234720,0.241388,0.217383,0.220050,0.232053,0.0,...,0.000000,0.000000,0.000000,0.000000,0.001564,0.001572,0.000000,0.000000,0.000000,0.000000


# Model Training section

##  MLP Classifier (trial)

In [111]:
model_mlp = MLPClassifier( hidden_layer_sizes=(100), activation= 'relu', max_iter=1000, alpha=0.0001,
                           solver= 'adam', tol=0.0001 ) #changed from logistic to relu for better output

In [231]:
model_mlp.fit(X_train_pca, y_train) #train the data using the MLP

MLPClassifier(hidden_layer_sizes=100, max_iter=1000)

In [232]:
y_pred = model_mlp.predict(X_test_pca) # predict the class of Test

In [233]:
accuracy_mlp = accuracy_score(y_test, y_pred) #accuracy of test data

In [234]:
accuracy_mlp

0.9779874213836478

## MLP Trial TFIDF

In [113]:
model_mlp.fit(X_train_tfidffeatures, y_train) #train the data using the MLP
y_pred = model_mlp.predict(X_test_tfidffeatures) # predict the class of Testa
accuracy_mlp = accuracy_score(y_test, y_pred) #accuracy of test data

accuracy_mlp

0.6519916142557652

## Entropy Trial

In [75]:
model_mlp.fit(X_train_entropy_array, y_train) #train the data using the MLP
y_pred = model_mlp.predict(X_test_entropy_array) # predict the class of Test
accuracy_mlp = accuracy_score(y_test, y_pred) #accuracy of test data
accuracy_mlp

0.5786163522012578

## SVM Classifier

In [235]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_pca, y_train)     # do the training
y_pred2 = model_svm.predict(X_test_pca) # work on the test data

KeyboardInterrupt: 

In [ ]:
accuracy_svm = accuracy_score(y_test, y_pred2) #accuracy of test data

In [ ]:
accuracy_svm

0.5943396226415094

## KNN Classifier

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 2)
model_knn.fit(X_train_pca,y_train)

KNeighborsClassifier(n_neighbors=2)

In [ ]:
#Predict Output
y_pred3 = model_knn.predict(X_test_pca) 

In [ ]:
accuracy_knn = accuracy_score(y_test, y_pred3) #accuracy of test data

In [ ]:
accuracy_knn

0.9245283018867925

## KNN Trial Entropy

In [129]:
model_knn_entropy = KNeighborsClassifier(n_neighbors = 4)
model_knn_entropy.fit(X_train_entropy_array,y_train)
#Predict Output
y_pred3 = model_knn_entropy.predict(X_test_entropy_array) 
accuracy_knn = accuracy_score(y_test, y_pred3) #accuracy of test data
accuracy_knn



0.9947589098532494

## CNN Classifier

In [ ]:
def cnn_evaluate(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 10, 32
	n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainy.shape[0]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [ ]:
# y_train_array = np.asarray (y_train)
# y_test_array = np.asarray (y_test)

X_train_cnn = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)


# cnn_accuracy = cnn_evaluate(X_train_pca.values, to_categorical(y_train_array), X_test_pca, to_categorical (y_test_array))

# cnn_accuracy

In [ ]:
print("X Train shape: ", X_train_cnn.shape)
print("X Test shape: ", X_test_cnn.shape)

X Train shape:  (2226, 4800, 1)
X Test shape:  (954, 4800, 1)


In [ ]:
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train_cnn.shape[1],1)))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPooling1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))
#Flatten the output
cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 2, activation='softmax'))



In [ ]:
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_31 (Conv1D)          (None, 4800, 32)          128       
                                                                 
 conv1d_32 (Conv1D)          (None, 4800, 64)          6208      
                                                                 
 conv1d_33 (Conv1D)          (None, 4800, 128)         24704     
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 2400, 128)        0         
 g1D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 2400, 128)         0         
                                                                 
 flatten_14 (Flatten)        (None, 307200)            0         
                                                     

In [ ]:
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)
cnn_model_history = cnn_model.fit(X_train_cnn, np.asarray (y_train), epochs=2, batch_size = 20 , validation_data = (X_test_cnn, np.asarray (y_test)))

Epoch 1/5
23/23 [==============================] - 68s 3s/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0021 - val_accuracy: 0.9990
Epoch 2/5
 3/23 [==>...........................] - ETA: 51s - loss: 6.9667e-04 - accuracy: 1.0000

KeyboardInterrupt: 

## K Means

In [32]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_pca)
y_labels_train_knn = kmeans_model.labels_
y_labels_test_knn = kmeans_model.predict(X_test_pca)

# X_train['km_clust'] = y_labels_train
# X_test['km_clust'] = y_labels_test


    

accuracy_kmeans = accuracy_score(y_train, y_labels_train_knn)

accuracy_kmeans = accuracy_score (y_test, y_labels_test_knn)

accuracy_kmeans 





0.5796645702306079

# Timepass

In [ ]:
x=real_df.values[0]

In [ ]:
x_ = attack_df.values[0]

In [ ]:
px.line(x[:1000])

In [ ]:
px.line(x_[:1000])

In [ ]:
z_ = scipy.io.loadmat('data/inputSample.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'data/inputSample.mat'

In [ ]:
z_['data'].shape

(1, 4800)

In [ ]:
a_ = scipy.io.loadmat('data/input_attack.mat')

In [ ]:
a_['data'].shape

(1, 4800)

In [ ]:
sd1 = [[1,2,3],[4,5,6]]
sd2 = [[7,8]]
sd1_df = pd.DataFrame(sd1)
sd2_df = pd.DataFrame(sd2)

In [ ]:
sd1_df.transpose()

,0,1
0,1,4
1,2,5
2,3,6


In [ ]:
sd2_df.transpose()

,0
0,7
1,8


In [ ]:
pd.concat([sd1_df.transpose(), sd2_df.transpose()], axis=0)

,0,1
0,1,4.0
1,2,5.0
2,3,6.0
0,7,NaN
1,8,NaN
